In [0]:
!pip install transformers
!pip install seqeval
!pip install tensorboardx
!pip install torchvision==0.4.0 -f https://download.pytorch.org/whl/cu100/torch_stable.html --no-cache-dir
!rm -rf .git
!rm -rf apex
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
!pip3 install simpletransformers
!pip install Afinn

In [0]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My\ Drive/Colab\ Notebooks

In [0]:
import numpy as np
import pandas as pd
import torch
import gc
import csv

from CsvReader import ReadCsv
from HelperFunctions import ShuffleData, replace_all, ReplaceSentimentsWithIndexes, SplitArticlesIntoSentiments, getMetrics, TransformDataIntoDataframe, CalculateWeights
from TrainEvalModel import TrainModelForMultiClass, EvalFromModel

data, categories, tendencies = ReadCsv('/content/drive/My Drive/Colab Notebooks Pascal/bert-etiki/etiki-data','test.csv','companies.csv', 'categories.csv','references.csv','tendencies.csv', 'topics.csv')
sentimentData = ReplaceSentimentsWithIndexes(data[:,[0,7,13]])
labels = np.unique(sentimentData[:,1])
weights = CalculateWeights(labels,sentimentData)

for i in range(1,5):
  for j in range(2,6):
    positiveArticles, negativeArticles, controversialArticles = SplitArticlesIntoSentiments(sentimentData)
    positiveArticles = ShuffleData(positiveArticles)
    negativeArticles = ShuffleData(negativeArticles)
    controversialArticles = ShuffleData(controversialArticles)

    trainData = np.vstack((positiveArticles[:int(len(positiveArticles)*0.7)],controversialArticles[:int(len(controversialArticles)*0.7)],negativeArticles[:int(len(negativeArticles)*0.7)]))
    testData = np.vstack((positiveArticles[int(len(positiveArticles)*0.7):],controversialArticles[int(len(controversialArticles)*0.7):],negativeArticles[int(len(negativeArticles)*0.7):]))
    train_df = TransformDataIntoDataframe(trainData)
    eval_df = TransformDataIntoDataframe(testData)
#------------------------------XLNet------------------------------
    algo = 'xlnet'        
    args = {'reprocess_input_data': True,
              'overwrite_output_dir': True,
              'num_train_epochs':j ,
              'silent':True,
              
              }

    model = TrainModelForMultiClass(algo, 'xlnet-base-cased',train_df,3,args)
    result, model_outputs, wrong_predictions = EvalFromModel(model,eval_df)

    with open('results/gradient/'+str(j)+'epoch/'+algo+'-mcc.csv', 'a', newline='') as f:
      writer = csv.writer(f)
      writer.writerow([result['mcc'],result["acc"],result["eval_loss"]])

    getMetrics(result["matr"],3,'gradient/'+str(j)+'epoch/'+algo)

  args = {'reprocess_input_data': True,
            'overwrite_output_dir': True,
            'num_train_epochs':4 ,
            'silent':True
            }

  model = TrainModelForMultiClass(algo, 'roberta-base',train_df,3,args,[1,1,1])
  result, model_outputs, wrong_predictions = EvalFromModel(model,eval_df)

  with open('results/weight/'+algo+'-noweight-mcc-'+'.csv', 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([result['mcc'],result["acc"],result["eval_loss"]])

  getMetrics(result["matr"],3,'weight/'+algo+'-noweight')
  torch.cuda.empty_cache()
  del model
  del result
  del model_outputs
  del wrong_predictions
  torch.cuda.empty_cache()
  gc.collect()